In [1]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs
import os
from string import Template

## sep變數：切割字元

def fmtViewColumns(r):
    sep, jsCol, colDscr, dataType, isOrderBy, lq ,rq = "\n\t\t", r["jsCol"], r["ColDscr"], r["Type"], r["spOrderBy"],"{", "}"
    dataType = "right" if (dataType in ["tinyint", "int", "smallint", "float", "decical"]) else "left"
    field = "(row) => dayjs(row.updateDT).format('YYYY-MM-DD HH:mm:ss')" if(jsCol == "updateDT") else "'" + jsCol + "'"

    tmp = f"""\n  $lq
    name: '$jsCol',
    label: '$colDscr',
    align: '$dataType',
    field: $field,
    sortable: true,
  $rq,"""

    tmp = Template(tmp)
    return tmp.substitute(sep = sep, jsCol = jsCol, colDscr= colDscr, dataType = dataType, field = field, lq = lq ,rq = rq) if(isOrderBy =="V") else ""

def fmtEditForm_q_Input(r):
    qinput, jsCol, colDscr, maxLen, _isNull, typ, vqInput, autoFocus = r["vue-q-input"], r["jsCol"], r["ColDscr"], r["maxLen"], r["isNull"], r["Type"], r["vue-q-input"], r["vue-autofocus"]
    isNull      = "" if (_isNull =="V") else "*"
    requirRule  = "" if (_isNull =="V") else "useRequiredInput, "
    ctrlTyp     = "number" if (typ in ["tinyint","int","smallint"]) else "text"
    vqInput     = "\n                autofocus" if (autoFocus == "V") else ""
    tmp =""
    
    tmpText = f"""
            <q-input{vqInput}
                class="col"
                type="{ctrlTyp}"
                v-model.trim="formData.{jsCol}"
                label="{isNull} {colDscr}"
                lazy-rules
                :rules="[{requirRule}(val) => useMaxLength(val, {maxLen})]"
            />"""

    tmpNumber = f"""
            <q-input{vqInput}
                class="col"
                type="{ctrlTyp}"
                v-model.trim="formData.{jsCol}"
                label="{isNull} {colDscr}"
                lazy-rules
                :rules="[useRequiredInput]"
            />"""

    tmpToggle=f"""
            <div class="col row items-center">
                <q-toggle
                    v-model="formData.{jsCol}"
                    color="primary"
                    left-label
                    :label="`{isNull} {colDscr}`"
                />
            </div>"""

    tmpDatePicker=f"""<q-input v-model="formData.{jsCol}" mask="date" :rules="['date']">
                <template v-slot:append>
                  <q-icon name="event" class="cursor-pointer">
                    <q-popup-proxy
                      ref="qDateProxy"
                      cover
                      transition-show="scale"
                      transition-hide="scale"
                    >
                      <q-date v-model="formData.{jsCol}">
                        <div class="row items-center justify-end">
                          <q-btn v-close-popup label="Close" color="primary" flat />
                        </div>
                      </q-date>
                    </q-popup-proxy>
                  </q-icon>
                </template>
            </q-input>"""
    
    if(typ in ["varchar","nvarchar"]):
        tmp = tmpText
    if(typ in ["tinyint","int","smallint"]):
        tmp = tmpNumber
    elif(typ in ["bit"]):
        tmp = tmpToggle
    elif(typ in ["date","datetime","datetime2"]):
        tmp = tmpDatePicker
    
    return "" if (qinput == "") else tmp.format(jsCol = jsCol, colDscr = colDscr, maxLen = maxLen, vqInput = vqInput)

def fmtEditForm_q_Input_IsEnable(r):
    col = r["Column"]
    tmp ="""<q-toggle
                v-model="formData.isEnable"
                color="primary"
                :label="formData.isEnable ? '啟用' : '停用'"
            />"""
    return tmp if (col == "IsEnable") else ""

    
def fmtJSColumn(r):
    col = r["Column"]
    return col[0:1].lower() + col[1:]

def fmtDotNetType(r):
    t, typ = r["Type"], ""
    if t in ["int","tinyint","smallint", "float","decimal"]:
        typ = "int"
    elif t in ["varchar","nvarchar","text"]:
        typ = "string"
    elif t in ["date","datetime","datetime2"]:
        typ = "DateTime"
    elif t == "bit":
        typ = "bool"
    return typ

def fmtJSType(r):
    t, typ = r["Type"], ""
    if t in ["int", "tinyint", "smallint", "float", "decimal"]:
        typ = "Integer"
    elif t in ["varchar","nvarchar","text"]:
        typ = "String"
    elif t in ["date","datetime","datetime2"]:
        typ = "DateTime"
    elif t == "bit":
        typ = "Boolean"
    return typ


def fmtApiModels(r):
    col, colDscr, typ, dotnetType, isNull = r["Column"], r["ColDscr"], r["Type"], r["dotnetType"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""
    isNull_b = " = null!;" if (isNull == "" and typ in ["varchar","nvarchar"]) else ""

    tmp = Template("""
        /// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType$isNull_a $col { get; set; } $isNull_b""")

    return tmp.substitute(col = col, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a, isNull_b = isNull_b)

def fmtApiModelsJs(r):
    jsCol, colDscr, typ, dotnetType, isNull = r["jsCol"], r["ColDscr"], r["Type"], r["dotnetType"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""
    isNull_b = " = null!;" if (isNull == "" and typ in ["varchar","nvarchar"]) else ""

    tmp = Template("""
        /// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType$isNull_a $jsCol { get; set; }""")

    return tmp.substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a, isNull_b = isNull_b)

def fmtDotNetDscr(r):
    jsCol, colDscr, dotnetType, col = r["jsCol"], r["ColDscr"], r["dotnetType"], r["Column"]
    jsCol = '{:<16}'.format(jsCol)
    dotnetType = '{:<10}'.format(dotnetType)

    tmp = Template("\t\t/// $jsCol - $dotnetType - $colDscr\n")
    return tmp.substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType)

def fmtDotNetDscr_Query(r):
    jsCol, colDscr, dotnetType, col = r["jsCol"], r["ColDscr"], r["dotnetType"], r["Column"]
    jsCol = '{:<16}'.format(jsCol)
    dotnetType = "string" if (col == "IsEnable") else dotnetType
    dotnetType = '{:<10}'.format(dotnetType)

    tmp = Template("\t\t/// $jsCol - $dotnetType - $colDscr\n")
    return tmp.substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType)


def fmtPkParasInput(r):
    jsCol, dotnetType, isNull = r["jsCol"], r["dotnetType"] , r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""
    return Template("$dotnetType$isNull_a $jsCol").substitute(jsCol = jsCol, dotnetType = dotnetType, isNull_a =isNull_a)

def fmtData2Json(r):
    sep, col, jsCol = "\t\t\t\t", r["Column"], r["jsCol"] 
    return Template("$sep$jsCol = data.$col,\n").substitute(sep = sep, col = col, jsCol = jsCol)

def fmtItem2Json(r):
    sep, col, jsCol = "\t\t\t\t", r["Column"], r["jsCol"]
    return Template("$sep$jsCol = item.$col,\n").substitute(sep = sep, col = col, jsCol = jsCol)

def fmtJson2Data(r):
    sep, col, jsCol = "\t\t\t\t\t", r["Column"], r["jsCol"]
    return Template("$sep$col = $jsCol,\n").substitute(sep = sep, col = col, jsCol = jsCol)

def fmtJson2Json(r):
    jsCol = r["jsCol"]
    return Template("$jsCol = $jsCol").substitute(jsCol = jsCol)

def fmtRequest2Json(r):
    jsCol = r["jsCol"]
    return Template("request.$jsCol").substitute(jsCol = jsCol)

def fmtInsertRequestItem(r):
    jsCol, dotnetType, colDscr, isNull = r["jsCol"], r["dotnetType"] , r["ColDscr"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""

    tmp ="""/// <summary>
    /// $colDscr
    /// </summary>
    public $dotnetType$isNull_a $jsCol { get; set; }"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a)

def fmtInsertValidator(r):
    tmp, tmp2, jsCol, colDscr, maxLen, isNull, dotnetType = "", "", r["jsCol"] ,r["ColDscr"], r["maxLen"], r["isNull"], r["dotnetType"]
     
    if (dotnetType == "string"):
        tmp  = "" if (isNull =="V") else """\t\t\tRuleFor(x => x.$jsCol).NotEmpty().WithMessage("$colDscr為必填");\n"""
        tmp2 = "" if (maxLen =="") else """\t\t\tRuleFor(x => x.$jsCol).MaximumLength($maxLen).WithMessage("$colDscr長度超過系統限制");\n""" 
        
    tt = tmp +tmp2
    return Template(tt).substitute(jsCol = jsCol, colDscr = colDscr, maxLen = maxLen)

def fmtValidateMustHave(r):
    jsCol, colDscr, maxLen = r["jsCol"] ,r["ColDscr"], r["maxLen"]
    tmp = """\t\t\tRuleFor(x => x.$jsCol).NotEmpty().WithMessage("$colDscr不能為空!!");\n"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, maxLen = maxLen)

def fmtQueryRequest(r):
    jsCol, dotnetType, colDscr, isNull, reqDefaultVal = r["jsCol"], r["dotnetType"] , r["ColDscr"], r["isNull"], r["reqDefaultVal"]
    # isNull_a = "?" if (isNull == "V") else ""
    reqDefaultVal = reqDefaultVal if (dotnetType in ["string","DateTime"]) else f" = \"{reqDefaultVal}\""

    dotnetType = dotnetType +"?" if (dotnetType in ["string","DateTime"]) else dotnetType
    dotnetType = "string?" if (jsCol =="isEnable") else dotnetType

    tmp ="""\t\t/// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType $jsCol { get; set; }\n"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, reqDefaultVal = reqDefaultVal)

def fmtQueryResponse(r):
    jsCol, dotnetType, colDscr, isNull = r["jsCol"], r["dotnetType"] , r["ColDscr"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""

    tmp ="""\t\t/// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType$isNull_a $jsCol { get; set; }\n"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a)

def fmtIsNullOrEmpyt(r):
    tmp, jsCol, dotnetType = "", r["jsCol"], r["dotnetType"]

    if(dotnetType == "string"):
        tmp ="\t\t\tif (string.IsNullOrEmpty(request.$jsCol)) { request.$jsCol = string.Empty; }\n"

    return Template(tmp).substitute(jsCol = jsCol)

def fmtJsActionCmmt(r):
    jsCol, jSType, dotnetTyp = r["jsCol"], r["jSType"], r["dotnetType"]
    jsCol = '{:<22}'.format(jsCol)
    tmp ="   * $jsCol    :  $jSType\n"
    return Template(tmp).substitute(jsCol = jsCol, jSType = jSType)

def fmtJsIndexVue_IsEnableSelect(r):
    tmp = """\n      <q-select class="filter-isenable" v-model="filter.isEnable" outlined dense :options="isEnableOptions" label="是否啟用" />"""
    return tmp

def fmtJsSupervisorRouteItems(r):
    tmp = """ {
    path: 'user',
    name: 'UserIndex',
    exact: true,
    meta: {
      title: '使用者維護',
    },
    component: () =>
      import(/* webpackChunkName: "supervisor-user" */ '@views/supervisor/user/index.vue'),
  },"""
    return tmp

def fmtEditFormIniValue(r):
    split, col, jsCol, type, val = "      ", r["Column"], r["jsCol"], r["Type"], ""
    if type in ["int", "tinyint", "smallint", "float","decimal"]:
        val = "0"
    elif type in ["varchar","nvarchar","text"]:
        val = "''"
    elif type in ["datetime2","datetime","date"]:
        val = "''"
    elif type in ["bit"]:
        val = "true"
    else:
        val="''"

    tmp = Template("$split$jsCol: $val,\n")
    return tmp.substitute(split = split, jsCol = jsCol, val = val)


def fmtViewFilterItem(r):
    sep, jsCol, colDscr, filterItem, lq, rq = "\n\t\t", r["jsCol"], r["ColDscr"], r["vueFilterItem"], "{", "}"
    tmp =f"""\n      $lq
        label: '$colDscr',
        value: '$jsCol',
        placeholder: '請輸入$colDscr',
      $rq,"""
    tmp = Template(tmp)
    return tmp.substitute(sep = sep ,jsCol = jsCol, colDscr= colDscr, lq = lq, rq = rq) if(filterItem =="V") else ""


In [10]:
from string import Template

_ExcelFile = "refFiles/TMP1.xlsx"
pd.options.mode.chained_assignment = None  # default='warn'
class MyTemplate(Template):
    delimiter = "//$"           #自己定義的替換字元符號

df_sp = pd.read_excel(_ExcelFile, sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')
df = pd.read_excel(_ExcelFile, sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')

df["jsCol"]             = df.apply(fmtJSColumn, axis =1)            # PcPhone 轉為 pcPhone 
df["indexVue_Columns"]  = df.apply(fmtViewColumns, axis = 1)        # {name:'extCode',label:'分機',align:left,field:'extCode',sor
df["filterItem2"]       = df.apply(fmtViewFilterItem, axis = 1)     #
df["defaultInitValue"]  = df.apply(fmtEditFormIniValue, axis = 1)
df["qInputStr"]         = df.apply(fmtEditForm_q_Input, axis = 1)
df["qInputIsEnableStr"] = df.apply(fmtEditForm_q_Input_IsEnable, axis = 1)
df["dotnetType"]        = df.apply(fmtDotNetType, axis = 1)          #Sql資料型別轉換為 .net資料型別
df["jSType"]            = df.apply(fmtJSType, axis = 1)
df["api_Models"]        = df.apply(fmtApiModels, axis = 1)
df["api_DotnetDscr"]    = df.apply(fmtDotNetDscr, axis = 1)          #.net內的欄位說明
df["api_DotnetDscr_Query"] = df.apply(fmtDotNetDscr_Query, axis = 1)       #.net內的欄位說明
df["api_PkParasInput"]  = df.apply(fmtPkParasInput, axis =1)
df["api_Data2Json"]     = df.apply(fmtData2Json, axis = 1)         #轉換為Json
df["api_Item2Json"]     = df.apply(fmtItem2Json, axis = 1)         #轉換為Json
df["api_Json2Data"]     = df.apply(fmtJson2Data, axis = 1)         #轉換為Json
df["api_Json2Json"]     = df.apply(fmtJson2Json, axis = 1)  
df["api_Request2Json"]  = df.apply(fmtRequest2Json, axis = 1)  
df["api_ApiModelsJs"]   = df.apply(fmtApiModelsJs, axis = 1)     #///<summary>分機</summary>public string? extCode { get;...   
df["api_InsertValidator"] = df.apply(fmtInsertValidator, axis = 1)
df["api_ValidateMustHave"] = df.apply(fmtValidateMustHave, axis = 1)
df["api_QueryRequest"]  = df.apply(fmtQueryRequest, axis = 1)
df["api_QueryResponse"] = df.apply(fmtQueryResponse, axis = 1)
df["isNullOrEmpyt"]     = df.apply(fmtIsNullOrEmpyt, axis =1)
df["jsActionCmmt"]      = df.apply(fmtJsActionCmmt, axis =1)
df["JsIndexVue_IsEnableSelect"] = df.apply(fmtJsIndexVue_IsEnableSelect, axis =1)
# df["JsSupervisorRouteItems"] = df.apply(fmtJsSupervisorRouteItems, axis =1)
   
## 通用函數：提供 source template file， target: 套表完成的路徑。
def genOneCode(source, target, _dictParas):
    _source = "template/" + source
    fileName = target
    # print("source:", _source)
    # print("target:", fileName)

    tmp = codecs.open(_source, mode="r",encoding="utf-8", buffering=-1).read()
    t = Template(tmp).substitute(_dictParas)
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t)
    f.close()

def copyJsFile(tbName,fName):
    tmp_editForm = codecs.open(f"template/JS/store/admin/{fName}.js",mode="r",encoding="utf-8", buffering=-1).read()
    t = MyTemplate(tmp_editForm).substitute(pt_TableName = tbName)

    tbName2 = "aux2" if(tbName == "aux") else tbName
    fileName = f"JS/src/store/supervisor/{tbName2}/{fName}.js"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t)
    f.close()

escape4C = ["Creator","CreateDT","CreatorName", "Updator", "UpdateDT", "UpdatorName"]   ## 排除掉4C

def genJSCode(tb, dfm, dfc):
    pt_TableName, pt_tableName, pt_table_name = dfm["TableName"].values[0], dfm["tableName"].values[0], dfm["table_name"].values[0]

    target = dfc[dfc["spOrderBy"] == "V"]
    target = target[ ~target["Column"].isin(["UpdateDT","IsEnable"]) ]

    pt_indexVue_Columns = "".join(dfc[~dfc["Column"].isin(escape4C)]["indexVue_Columns"].tolist())[3:]
    pt_filterItem = "".join(target["filterItem2"].tolist())[1:]
    pt_editForm_formData = "".join(dfc[~dfc["Column"].isin(escape4C)]["defaultInitValue"].tolist())[:-1]

    df_q = dfc[dfc["vue-q-input"] != ""]
    # print(tb, df_q[df_q.notna()]["vue-q-input"].astype(int).max())
    # print(tb, df_q)

    iMax = int(df_q["vue-q-input"].max())
    pt_EditForm_q_input =""
    for q in range(iMax):
        k, ss = q+1, ""
        for e in df_q[df_q["vue-q-input"]==k]["qInputStr"].tolist():
            ss += e

        pt_EditForm_q_input += f"""
            <div class="row q-col-gutter-md">{ss}
            </div>""".format(ss = ss)
    
    pt_EditForm_q_input = pt_EditForm_q_input[1:]
    pt_EditForm_q_input_isEnable = "".join(dfc["qInputIsEnableStr"].tolist())[:-1]
    pt_jsActionCmmt              = "".join(dfc[(dfc["spInsert"] !="") & (dfc["Column"]!="SeqNo")]["jsActionCmmt"].tolist())[:-1]
    pt_JsIndexVue_IsEnableSelect ="".join(dfc[(dfc["jsCol"]=="isEnable")]["JsIndexVue_IsEnableSelect"].tolist())
    # print(pt_EditForm_q_input)
    
    apiUrlPrefix ='/api/'+pt_TableName

    jsDict =dict(pt_TableName = pt_TableName, pt_tableName = pt_tableName, pt_table_name = pt_table_name, pt_indexVue_Columns = pt_indexVue_Columns, pt_filterItem = pt_filterItem, pt_editForm_formData = pt_editForm_formData, pt_EditForm_q_input = pt_EditForm_q_input, apiUrlPrefix = apiUrlPrefix, pt_EditForm_q_input_isEnable = pt_EditForm_q_input_isEnable, pt_jsActionCmmt =pt_jsActionCmmt, pt_JsIndexVue_IsEnableSelect =pt_JsIndexVue_IsEnableSelect )
    
    # print(pt_table_name)
    pt_table_name2 = "aux2" if (pt_table_name == "aux") else pt_table_name   ###
    ## -----套表區
    genOneCode(f"JS/views/admin/index.vue",     f"JS/src/views/supervisor/{pt_table_name2}/index.vue", jsDict)
    genOneCode(f"JS/views/admin/edit-form.vue", f"JS/src/views/supervisor/{pt_table_name2}/edit-form.vue", jsDict)  
    genOneCode(f"JS/store/admin/actions.js",    f"JS/src/store/supervisor/{pt_table_name2}/actions.js", jsDict)  
    copyJsFile(pt_table_name, "getters")
    copyJsFile(pt_table_name, "index")
    copyJsFile(pt_table_name, "mutations")
    copyJsFile(pt_table_name, "state")

_sq = "SeqNo"

def genWebApiCode(tb, dfm, dfc):
    pt_TableName, pt_tableName, pt_table_name = dfm["TableName"].values[0], dfm["tableName"].values[0], dfm["table_name"].values[0]
    TbDscr = dfm["tbDscr"].values[0]

    pt_apiModels = "".join(dfc[ ~dfc["Column"].isin(escape4C) ]["api_Models"].tolist())
    pt_apiModels_insertRequest = "".join(dfc[dfc["spInsert"] != ""]["api_ApiModelsJs"].tolist())[1:]
    pt_InputPK          = ",".join(dfc[dfc["Column"] == _sq]["api_PkParasInput"].tolist())
    pt_InputServicePK   = ",".join(dfc[dfc["Column"] == _sq]["jsCol"].tolist())
    pt_ColDscr_ParasPK  = "".join(dfc[dfc["Column"] == _sq]["api_DotnetDscr"].tolist())[:-1]

    pt_InputIsExist = ", ".join(dfc[dfc["spExist"] != ""]["api_PkParasInput"].tolist())

    
    pt_ColDscr_GetReturnAll = "".join(dfc["api_DotnetDscr"].tolist())[:-1]
    pt_ColDscr_QueryParas = "".join(dfc[dfc["spInsert"] != ""]["api_DotnetDscr"].tolist())[:-1]
    pt_ColDscr_QueryParas_Query = "".join(dfc[dfc["spQueryWhere"] != ""]["api_DotnetDscr_Query"].tolist())[:-1]
    pt_ColDscr_InsertParas = "".join(dfc[dfc["spInsert"] != ""]["api_DotnetDscr"].tolist())[:-1]
    pt_colDscr_UpdateParas = "".join(dfc[dfc["spUpdate"] != ""]["api_DotnetDscr"].tolist())[:-1]
    pt_colDscr_ExistsParas = "".join(dfc[dfc["spExist"] != ""]["api_DotnetDscr"].tolist())[:-1]
    
    pt_data2Json = "".join(dfc["api_Data2Json"].tolist())[:-1]
    pt_item2Json = "".join(dfc["api_Item2Json"].tolist())[:-1]
    pt_json2Data = "".join(dfc[dfc["spExist"] != ""]["api_Json2Data"].tolist())[:-1]
    pt_json2json = ", ".join(dfc[dfc["PK"] == "V"]["api_Json2Json"].tolist())
    pt_request2Json = ", ".join(dfc[dfc["spExist"] != ""]["api_Request2Json"].tolist())

    pt_insertValidator = "".join(dfc[(dfc["Type"].isin(["varchar","nvarchar","bit"])) & (~dfc["Column"].isin(escape4C)) ]["api_InsertValidator"].tolist())[:-1]
    pt_ValidateMustHave ="".join(dfc[ dfc["PK"] == "V"]["api_ValidateMustHave"].tolist())[:-1]
    pt_queryRequest = "".join(dfc[dfc["spQueryWhere"] != ""]["api_QueryRequest"].tolist())
    pt_queryResponse = "".join(dfc[dfc["spQueryWhere"] != ""]["api_QueryResponse"].tolist())[:-1]
    pt_updateRequest = "".join(dfc[dfc["spUpdate"] != ""]["api_QueryResponse"].tolist())[:-1]
    pt_requestIsNullOrEmpty ="".join(dfc[(dfc["spQueryWhere"] !="") & (dfc["Column"]!="IsEnable")]["isNullOrEmpyt"].tolist())
    pt_requstInsertIsExist = ", ".join(dfc[(dfc["spExist"] !="" ) & (dfc["Column"]!="SeqNo")]["api_Request2Json"].tolist())
    pt_requstInsertIsExistWithSeqNo = ", ".join(dfc[(dfc["spExist"] !="")]["api_Request2Json"].tolist())
    
    # print(pt_JsIndexVue_IsEnableSelect)

    ## 準備要套用變數。
    dictParas = dict(pt_TableName = pt_TableName, TbDscr = TbDscr, controller="Controller", service="Service", queryRequest = "QueryRequest", insertRequest="InsertRequest", insertValidator = "InsertValidator", updateRequest = "UpdateRequest", updateValidator = "UpdateValidator", get = "Get", response ="Response", request ="Request", exists = "Exists", query = "Query", insert = "Insert", update = "Update", delete = "Delete", pt_apiModels = pt_apiModels, pt_ColDscr_ParasPK = pt_ColDscr_ParasPK, pt_InputPK = pt_InputPK, pt_InputServicePK = pt_InputServicePK,  pt_ColDscr_GetReturnAll = pt_ColDscr_GetReturnAll , pt_ColDscr_QueryParas = pt_ColDscr_QueryParas, pt_ColDscr_InsertParas = pt_ColDscr_InsertParas, pt_colDscr_UpdateParas = pt_colDscr_UpdateParas, pt_colDscr_ExistsParas = pt_colDscr_ExistsParas, pt_InputIsExist = pt_InputIsExist , pt_data2Json = pt_data2Json, pt_item2Json = pt_item2Json, pt_json2Data = pt_json2Data, pt_json2json = pt_json2json, pt_request2Json = pt_request2Json, pt_apiModels_insertRequest = pt_apiModels_insertRequest, pt_insertValidator = pt_insertValidator, pt_ValidateMustHave = pt_ValidateMustHave, pt_queryRequest = pt_queryRequest, pt_queryResponse = pt_queryResponse, pt_updateRequest = pt_updateRequest, pt_ColDscr_QueryParas_Query = pt_ColDscr_QueryParas_Query, pt_requestIsNullOrEmpty = pt_requestIsNullOrEmpty, pt_requstInsertIsExist = pt_requstInsertIsExist, pt_requstInsertIsExistWithSeqNo = pt_requstInsertIsExistWithSeqNo)

    pt_TableName2 = "Aux2" if (pt_TableName == "Aux") else pt_TableName   ###

    ## 開始套表...
    genOneCode("WebApi/Models/TbPcPhone.cs",                             f"WebApi/Models/Tb{pt_TableName}.cs", dictParas)
    genOneCode("WebApi/Controllers/PcPhoneController.cs",                f"WebApi/Controllers/{pt_TableName}Controller.cs", dictParas)
    genOneCode("WebApi/Application/IPcPhoneService.cs",                  f"WebApi/Application/{pt_TableName2}/I{pt_TableName}Service.cs", dictParas)
    genOneCode("WebApi/Application/PcPhoneService.cs",                   f"WebApi/Application/{pt_TableName2}/{pt_TableName}Service.cs", dictParas)
    genOneCode("WebApi/Application/Contract/PcPhoneInsertRequet.cs",     f"WebApi/Application/{pt_TableName2}/Contract/{pt_TableName}InsertRequest.cs", dictParas)
    genOneCode("WebApi/Application/Contract/PcPhoneInsertValidator.cs",  f"WebApi/Application/{pt_TableName2}/Contract/{pt_TableName}InsertValidator.cs", dictParas) 
    genOneCode("WebApi/Application/Contract/PcPhoneQueryRequest.cs",     f"WebApi/Application/{pt_TableName2}/Contract/{pt_TableName}QueryRequest.cs", dictParas) 
    genOneCode("WebApi/Application/Contract/PcPhoneResponse.cs",         f"WebApi/Application/{pt_TableName2}/Contract/{pt_TableName}Response.cs", dictParas) 
    genOneCode("WebApi/Application/Contract/PcPhoneUpdateRequest.cs",    f"WebApi/Application/{pt_TableName2}/Contract/{pt_TableName}UpdateRequest.cs", dictParas) 
    genOneCode("WebApi/Application/Contract/PcPhoneUpdateValidator.cs",  f"WebApi/Application/{pt_TableName2}/Contract/{pt_TableName}UpdateValidator.cs", dictParas) 
 
tbs = df_sp[df_sp["isCopy"]=="V"]["TableName"].unique()
print(tbs)

for tb in tbs:
    dfm = df_sp[df_sp["TableName"] == tb]
    dfc = df[df["TableName"] == tb]

    genJSCode(tb, dfm, dfc)
    genWebApiCode(tb, dfm, dfc)



['DialBook' 'CountryCode' 'AuthCallReason' 'CTIStatus' 'ANICode'
 'SkillCode' 'IVRCode' 'ReviewType' 'BussinessUnit' 'BussinessB03Type'
 'KYCCode' 'CBP83Note' 'BranchCode' 'NotificationType' 'BankCardCode'
 'DebitCardCode' 'LossScript' 'EnterpriseEmail' 'AccidentFormTemplate'
 'AccidentScript' 'MarketingControl' 'TQMSetting' 'ImportScore'
 'LevelScript' 'ImportSchedule' 'ImportOvertime' 'MealScript' 'SysConfig']
